# 📊 Jour 6 : Régression Linéaire Multiple avec `statsmodels`

La régression multiple est une extension de la régression linéaire simple qui permet d'expliquer une variable dépendante à partir de plusieurs variables explicatives.
Dans ce cours, nous allons utiliser plusieurs prédicteurs pour expliquer le `tip` dans le dataset `tips`.

In [ ]:
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt

## 📥 Chargement du dataset

In [ ]:
df = sns.load_dataset('tips')
df.head()

## 🔄 Encodage des variables catégorielles

In [ ]:
# On encode les variables catégorielles avec des variables fictives
df_encoded = pd.get_dummies(df, drop_first=True)
df_encoded.head()

## 📈 Implémentation de la régression multiple

In [ ]:
X = df_encoded.drop(columns='tip')
y = df_encoded['tip']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
model.summary()

## 📊 Analyse des résultats

In [ ]:
# On affiche les coefficients significatifs
model.params[model.pvalues < 0.05]

## 📝 Exercices
- Quelle est la variable la plus significative dans la prédiction du pourboire ?
- Essayez un modèle uniquement avec `total_bill`, `size`, et `sex_Male`.
- Visualisez les résidus du modèle final.
- Vérifiez la multicolinéarité avec un `heatmap` des corrélations.